In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sat Dec  2 15:21:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q datasets seqeval
!pip install gradio
!sudo apt install tesseract-ocr
!pip install pytesseract


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381

In [ ]:
!pip freeze


In [ ]:
# UNZIP THE FOLDER
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/dataExtraction.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
path = '/content/Data Extraction'

In [ ]:
import pandas as pd
df = pd.read_csv(path + "/DE-TRAIN.csv")
test_ds = pd.read_csv(path + "/DE-TEST.csv")

In [ ]:
!mkdir '/content/Data Extraction/Train Images'
!mkdir '/content/Data Extraction/Masked Train Images'

In [ ]:
# CONVERTING PDF TO IMAGE
!pip install pdf2image
!apt-get install poppler-utils

import os
from pdf2image import convert_from_path

BASE_PDF_DIR = path + '/Train Files'
BASE_IMG_DIR = path + '/Train Images'
pdfs = os.listdir( path + "/Train Files")

for pdf in pdfs:
  print(f'{BASE_PDF_DIR}/{pdf}')
  images = convert_from_path(f'{BASE_PDF_DIR}/{pdf}')

  for i in range(len(images)):
    images[i].save(BASE_IMG_DIR + '/' + pdf.rsplit('.')[0] +'.jpg', 'JPEG')

In [ ]:
# MASKING THE UNWANTED CONTENTS FROM THE IMAGE
!pip install opencv-python
import numpy as np

import cv2
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

path = '/content/Data Extraction'
BASE_IMG_DIR = path + '/Masked Train Images'


def mask_two(image_name):
  # Define the dimensions of the center rectangle
  image_path = path +"/Train Images/" +image_name
  image = cv2.imread(image_path)

  image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

  # Define the coordinates of the rectangle to be masked
  x, y, width, height = 100, 50, 200, 150

  # Create a mask of the same size as the image
  mask = np.zeros_like(gray_image)
  x, y, width, height = 30, 300, 1200, 70

  # Draw a filled white rectangle on the mask
  cv2.rectangle(mask, (x, y), (x + width, y + height), (255, 255, 255), thickness=cv2.FILLED)


  x, y, width, height = 1330, 1000, 300, 1000

  # Draw a filled white rectangle on the mask
  cv2.rectangle(mask, (x, y), (x + width, y + height), (255, 255, 255), thickness=cv2.FILLED)
  # Bitwise AND operation to apply the mask to the image
  masked_img = cv2.bitwise_and(gray_image, mask)



  mask_folder = os.path.join(path, "Masked Train Images")
  os.makedirs(mask_folder, exist_ok=True)  # Create the "mask" folder if it doesn't exist
  masked_image_path = os.path.join(mask_folder, image_name)

  cv2.imwrite(masked_image_path, masked_img)


In [ ]:
# CONVERTING LABEL TO NUMERICAL VALUES
labels = list(df['label'].unique())
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}

def normalize_bbox(bbox, width, height):
    return [
        int(1000 * (bbox[0] / width)),
        int(1000 * (bbox[1] / height)),
        int(1000 * (bbox[2] / width)),
        int(1000 * (bbox[3] / height)),
    ]


datas = []
filenames = os.listdir(path + "/Train Images")
for filename, i in zip(filenames, range(len(filenames))):
  d = {}
  d['id'] = i
  d['words'] = []
  d['bboxes'] = []
  d['ner_tags'] = []
  d['image_path'] = f'/content/Data Extraction/Train Images/{filename}'
  items = df[df['file_name'] == f'{filename.split(".")[0]}.pdf']
  for j in range(len(items)):
    item = items.iloc[j]
    d['words'].append(item['value'])
    d['bboxes'].append(normalize_bbox([item['x0'], item['y0'], item['x2'], item['y2']], 1700, 2200))
    d['ner_tags'].append(label2id[item['label']])
  datas.append(d)



In [ ]:
dataset = pd.DataFrame(datas)

In [ ]:
dataset

,id,words,bboxes,ner_tags,image_path
0,0,"[Benjamin, Garcia, 96, 4744, 60600.262, 67, 50...","[[65, 149, 376, 164], [396, 148, 740, 163], [8...","[0, 1, 12, 22, 2, 15, 17, 23, 4, 5, 6, 16, 9, 11]",/content/Data Extraction/Train Images/1040_fil...
1,1,"[John, Lowery, 82361.474, 69945.85, 6284.977, ...","[[65, 149, 376, 164], [396, 148, 740, 163], [8...","[0, 1, 12, 24, 13, 14, 23, 3, 5, 16, 7, 18, 11]",/content/Data Extraction/Train Images/1040_fil...
2,2,"[Monica, Roberts, 7, 3850, 3926, 60369.91, 6, ...","[[65, 149, 376, 164], [396, 148, 740, 163], [8...","[0, 1, 12, 24, 13, 17, 23, 20, 4, 18, 10, 19]",/content/Data Extraction/Train Images/1040_fil...
3,3,"[Mrs., Roman, 82990.94, 12227.434, 1701, 5703,...","[[65, 149, 376, 164], [396, 148, 740, 163], [8...","[0, 1, 12, 24, 2, 17, 5, 16, 7, 18, 10, 21, 11]",/content/Data Extraction/Train Images/1040_fil...
4,4,"[Raymond, Friedman, 53403.59, 10111.38, 7, 83,...","[[65, 149, 376, 164], [396, 148, 740, 163], [8...","[0, 1, 24, 13, 22, 2, 15, 23, 9, 10]",/content/Data Extraction/Train Images/1040_fil...
...,...,...,...,...,...
155,155,"[Edward, Williams, 5108, 98449.76, 1403, 3842....","[[65, 149, 376, 164], [396, 148, 740, 163], [8...","[0, 1, 24, 13, 2, 17, 23, 3, 5, 7, 18, 19, 21]",/content/Data Extraction/Train Images/1040_fil...
156,156,"[Crystal, Hunt, 88210.273, 102, 7, 695, 24, 82...","[[65, 149, 376, 164], [396, 148, 740, 163], [8...","[0, 1, 24, 17, 3, 5, 6, 16, 8, 18, 19, 11]",/content/Data Extraction/Train Images/1040_fil...
157,157,"[Becky, Bryant, 74778.08, 16, 84785.95, 17491....","[[65, 149, 376, 164], [396, 148, 740, 163], [8...","[0, 1, 13, 22, 14, 2, 15, 23, 3, 4, 6, 7, 8, 1...",/content/Data Extraction/Train Images/1040_fil...
158,158,"[Andrew, Bell, 3, 98580.274, 49918.99, 697, 34...","[[65, 149, 376, 164], [396, 148, 740, 163], [8...","[0, 1, 24, 22, 14, 2, 5, 7, 9, 10, 19, 11]",/content/Data Extraction/Train Images/1040_fil...


In [ ]:
df_shuffled = dataset.sample(frac=1, random_state=42)


In [ ]:
df_shuffled.head()

,id,words,bboxes,ner_tags,image_path
105,105,"[Maureen, Hoffman, 23287.39, 8, 6, 366, 2268.2...","[[65, 149, 376, 164], [396, 148, 740, 163], [8...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",/content/Data Extraction/Train Images/1040_fil...
108,108,"[Mrs., Morrison, 42, 3959.14, 32217.45, 96129....","[[65, 149, 376, 164], [396, 148, 740, 163], [8...","[0, 1, 12, 14, 2, 15, 17, 4, 5, 6, 18, 9, 10, ...",/content/Data Extraction/Train Images/1040_fil...
141,141,"[Richard, Pollard, 92299.43, 61947, 62396.35, ...","[[65, 149, 376, 164], [396, 148, 740, 163], [8...","[0, 1, 12, 24, 22, 14, 2, 23, 20, 3, 4, 16, 7,...",/content/Data Extraction/Train Images/1040_fil...
55,55,"[Kaylee, Spence, 98988, 73532.239, 11387.52, 7...","[[65, 149, 376, 164], [396, 148, 740, 163], [8...","[0, 1, 22, 14, 2, 15, 23, 5, 6, 7, 8, 18, 10, ...",/content/Data Extraction/Train Images/1040_fil...
94,94,"[Brandon, Martin, 41460.62, 6009, 7093, 5203.6...","[[65, 149, 376, 164], [396, 148, 740, 163], [8...","[0, 1, 12, 14, 2, 15, 17, 20, 6, 16, 7, 9, 10,...",/content/Data Extraction/Train Images/1040_fil...


In [ ]:
# SPLITTING UP THE TRAIN SET INTO TRAIN AND TEST SET
frac_part1 = 0.8  # Fraction of data for the first part
train_set = df_shuffled.sample(frac=frac_part1, random_state=42)
test_set = df_shuffled.drop(train_set.index)


In [ ]:
import datasets
from datasets import Dataset
train_dataset = Dataset.from_dict(train_set)
test_dataset = Dataset.from_dict(test_set)
dataset = datasets.DatasetDict({"train":train_dataset, "test":test_dataset})

In [ ]:
# INSPECTION
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
        num_rows: 128
    })
    test: Dataset({
        features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
        num_rows: 32
    })
})

In [ ]:
from transformers import AutoProcessor

# we'll use the Auto API here - it will load LayoutLMv3Processor behind the scenes,
# based on the checkpoint we provide from the hub
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base",
                                          apply_ocr=False)                      # INITIALIZING THE PRETRAINED MODEL

preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
# UTILITY FUNCTIONS
from datasets.features import ClassLabel

features = dataset["train"].features
column_names = dataset["train"].column_names
image_column_name = "image_path"
text_column_name = "words"
boxes_column_name = "bboxes"
label_column_name = "ner_tags"

# In the event the labels are not a `Sequence[ClassLabel]`, we will need to go through the dataset to get the
# unique labels.
def get_label_list(labels):
    unique_labels = set()
    for label in labels:
        unique_labels = unique_labels | set(label)
    label_list = list(unique_labels)
    label_list.sort()
    return label_list

if isinstance(features[label_column_name].feature, ClassLabel):
    label_list = features[label_column_name].feature.names
    # No need to convert the labels since they are already ints.
    id2label = {k: v for k,v in enumerate(label_list)}
    label2id = {v: k for k,v in enumerate(label_list)}
else:
    label_list = get_label_list(dataset["train"][label_column_name])
    id2label = {k: v for k,v in enumerate(label_list)}
    label2id = {v: k for k,v in enumerate(label_list)}
num_labels = len(label_list)

In [ ]:
from PIL import Image


In [ ]:
def prepare_examples(examples):
  images = [Image.open(path).convert("RGB") for path in examples['image_path']] #Image.open(examples[image_column_name])
  words = examples[text_column_name]
  boxes = examples[boxes_column_name]
  word_labels = examples[label_column_name]

  encoding = processor(images, words, boxes=boxes, word_labels=word_labels,
                       truncation=True, padding="max_length")

  return encoding

In [ ]:
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D

# we need to define custom features for `set_format` (used later on) to work properly
features = Features({
    'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(ClassLabel(names=label_list)),
})

train_dataset = dataset["train"].map(
    prepare_examples,
    batched=True,
    remove_columns=column_names,
    features=features,
)
eval_dataset = dataset["test"].map(
    prepare_examples,
    batched=True,
    remove_columns=column_names,
    features=features,
)



Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [ ]:
train_dataset


Dataset({
    features: ['pixel_values', 'input_ids', 'attention_mask', 'bbox', 'labels'],
    num_rows: 128
})

In [ ]:
# INSPECTION
example = train_dataset[0]
processor.tokenizer.decode(example["input_ids"])

'<s> Jessica Sharp 61367.807 81624 17111.671 40 48293.639 26122 504 1918 3663 66</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [ ]:
train_dataset.set_format("torch")

In [ ]:
import torch

example = train_dataset[0]
for k,v in example.items():
    print(k,v.shape)

pixel_values torch.Size([3, 224, 224])
input_ids torch.Size([512])
attention_mask torch.Size([512])
bbox torch.Size([512, 4])
labels torch.Size([512])


In [ ]:
# INSPECTION
eval_dataset

Dataset({
    features: ['pixel_values', 'input_ids', 'attention_mask', 'bbox', 'labels'],
    num_rows: 32
})

In [ ]:
# INSPECTION
for id, label in zip(train_dataset[0]["input_ids"], train_dataset[0]["labels"]):
  print(processor.tokenizer.decode([id]), label.item())

<s> -100
 Jessica 0
 Sharp 1
 6 13
13 -100
67 -100
. -100
807 -100
 8 14
16 -100
24 -100
 17 2
111 -100
. -100
671 -100
 40 5
 48 6
293 -100
. -100
6 -100
39 -100
 26 16
122 -100
 504 7
 1918 8
 366 18
3 -100
 66 19
</s> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad

In [ ]:
from datasets import load_metric

metric = load_metric("seqeval")

<ipython-input-30-edef154daec7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
import numpy as np

return_entity_level_metrics = False

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [ ]:
from transformers import LayoutLMv3ForTokenClassification

model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base",
                                                         id2label=id2label,
                                                         label2id=label2id)

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# #Uncomment, Run, Restart Session, Comment the following lines
# ! pip install -U accelerate
# ! pip install -U transformers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="layoutlmv3-finetuned-invoice",
                                  max_steps=400,
                                  per_device_train_batch_size=2,
                                  per_device_eval_batch_size=2,
                                  learning_rate=1e-5,
                                  evaluation_strategy="steps",
                                  eval_steps=100,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1",
                                  push_to_hub=True,
                                  hub_model_id=f"layoutlmv3-finetuned-invoice", # this is the name we'll use for our model on the hub
                                  )


In [ ]:

from transformers.data.data_collator import default_data_collator

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
100,No log,0.018896,1.000000,1.000000,1.000000,1.000000
200,No log,0.012657,1.000000,1.000000,1.000000,1.000000
300,No log,0.010345,1.000000,1.000000,1.000000,1.000000
400,No log,0.009662,1.000000,1.000000,1.000000,1.000000


TrainOutput(global_step=400, training_loss=0.029155104160308837, metrics={'train_runtime': 322.3659, 'train_samples_per_second': 2.482, 'train_steps_per_second': 1.241, 'total_flos': 212369731584000.0, 'train_loss': 0.029155104160308837, 'epoch': 6.25})

In [ ]:
# SAVING THE TRAINER
trainer.save_model("/content/drive/MyDrive/KU")


In [ ]:
trainer.evaluate()


{'eval_loss': 0.009662076830863953,
 'eval_precision': 1.0,
 'eval_recall': 1.0,
 'eval_f1': 1.0,
 'eval_accuracy': 1.0,
 'eval_runtime': 5.6268,
 'eval_samples_per_second': 5.687,
 'eval_steps_per_second': 2.844,
 'epoch': 6.25}